# 企业知识库搭建
> By SherryAGI | 完整的企业级 RAG 系统

## 企业知识库架构

```
┌─────────────────────────────────────────────────────────────┐
│                    企业知识库系统                             │
├─────────────────────────────────────────────────────────────┤
│  📥 数据源                                                   │
│  ├── 文档库 (Confluence, SharePoint)                        │
│  ├── 数据库 (MySQL, PostgreSQL)                             │
│  ├── 聊天记录 (Slack, Teams)                                │
│  └── 代码库 (GitHub, GitLab)                                │
├─────────────────────────────────────────────────────────────┤
│  🔄 处理层                                                   │
│  ├── 文档解析 → 分块 → 向量化                                │
│  └── 增量更新 & 版本管理                                     │
├─────────────────────────────────────────────────────────────┤
│  💾 存储层                                                   │
│  ├── 向量数据库 (pgvector / Milvus)                         │
│  └── 元数据存储 (PostgreSQL)                                │
├─────────────────────────────────────────────────────────────┤
│  🔍 检索层                                                   │
│  ├── 混合检索 (向量 + 关键词)                                │
│  └── 重排序 (Reranker)                                      │
├─────────────────────────────────────────────────────────────┤
│  🤖 生成层                                                   │
│  ├── LLM (GPT-4 / Claude / 本地模型)                        │
│  └── 答案合成 & 来源引用                                     │
├─────────────────────────────────────────────────────────────┤
│  🖥️ 应用层                                                   │
│  ├── Web UI / Chatbot                                       │
│  ├── API 接口                                               │
│  └── 集成 (Slack, Teams, 企业微信)                          │
└─────────────────────────────────────────────────────────────┘
```

In [ ]:
import json
from datetime import datetime
from pathlib import Path
import numpy as np

print('🚀 企业知识库系统 by SherryAGI')

## 1. 知识库核心类

In [ ]:
class EnterpriseKnowledgeBase:
    """企业知识库 by SherryAGI"""
    
    def __init__(self, name="企业知识库"):
        self.name = name
        self.documents = []
        self.embeddings = []
        self.categories = set()
        self.stats = {
            "total_docs": 0,
            "total_chunks": 0,
            "last_updated": None
        }
    
    def _embed(self, text):
        """向量化（实际使用时替换为真实模型）"""
        np.random.seed(hash(text) % 2**32)
        return np.random.rand(384).tolist()
    
    def add_document(self, content, metadata=None):
        """添加文档"""
        doc = {
            "id": len(self.documents),
            "content": content,
            "metadata": metadata or {},
            "created_at": datetime.now().isoformat(),
            "embedding": self._embed(content)
        }
        
        self.documents.append(doc)
        self.embeddings.append(doc["embedding"])
        
        if metadata and "category" in metadata:
            self.categories.add(metadata["category"])
        
        self.stats["total_docs"] += 1
        self.stats["last_updated"] = datetime.now().isoformat()
        
        return doc["id"]
    
    def search(self, query, top_k=5, category=None):
        """搜索知识库"""
        from sklearn.metrics.pairwise import cosine_similarity
        
        query_embedding = self._embed(query)
        
        # 计算相似度
        similarities = cosine_similarity([query_embedding], self.embeddings)[0]
        
        # 过滤类别
        results = []
        for i, (sim, doc) in enumerate(zip(similarities, self.documents)):
            if category and doc["metadata"].get("category") != category:
                continue
            results.append((sim, doc))
        
        # 排序
        results.sort(key=lambda x: -x[0])
        
        return [
            {
                "score": float(sim),
                "content": doc["content"],
                "metadata": doc["metadata"],
                "id": doc["id"]
            }
            for sim, doc in results[:top_k]
        ]
    
    def ask(self, question, top_k=3):
        """问答接口"""
        # 检索相关文档
        results = self.search(question, top_k)
        
        # 构建上下文
        context = "\n\n".join([
            f"[来源: {r['metadata'].get('source', '未知')}]\n{r['content']}"
            for r in results
        ])
        
        # 构建 Prompt
        prompt = f"""基于以下企业知识库内容回答问题。请引用来源。

知识库内容：
{context}

问题：{question}

答案："""
        
        return {
            "question": question,
            "prompt": prompt,
            "sources": [r["metadata"].get("source", "未知") for r in results],
            "context_length": len(context)
        }
    
    def get_stats(self):
        """获取统计信息"""
        return {
            **self.stats,
            "categories": list(self.categories),
            "category_count": len(self.categories)
        }

print('✅ EnterpriseKnowledgeBase 已定义')

## 2. 初始化知识库

In [ ]:
# 创建知识库
kb = EnterpriseKnowledgeBase("SherryAGI 企业知识库")

# 添加 HR 政策
hr_docs = [
    ("年假政策：入职满1年享有5天年假，每增加1年加1天，上限15天。年假需提前申请，不可跨年累积。", 
     {"source": "HR手册", "category": "HR", "tags": ["年假", "休假"]}),
    ("病假政策：需提供医院证明，每年累计不超过30天。超过3天需HR审批。",
     {"source": "HR手册", "category": "HR", "tags": ["病假", "休假"]}),
    ("报销标准：差旅费经济舱机票、三星级酒店；餐饮费每日上限100元；500元以上需主管审批。",
     {"source": "财务制度", "category": "财务", "tags": ["报销", "差旅"]}),
]

# 添加 IT 支持
it_docs = [
    ("密码重置：方法1-自助重置（访问 reset.company.com）；方法2-联系IT热线8888。",
     {"source": "IT手册", "category": "IT", "tags": ["密码", "账号"]}),
    ("VPN连接：下载客户端，使用工号登录，选择最近服务器。遇到问题联系IT。",
     {"source": "IT手册", "category": "IT", "tags": ["VPN", "远程"]}),
    ("设备申请：通过OA系统提交申请，需部门主管审批，IT部门3个工作日内处理。",
     {"source": "IT手册", "category": "IT", "tags": ["设备", "申请"]}),
]

# 添加产品知识
product_docs = [
    ("产品A功能：支持数据分析、报表生成、自动化处理。适用于中大型企业。",
     {"source": "产品文档", "category": "产品", "tags": ["产品A", "功能"]}),
    ("产品A定价：基础版999元/月，专业版2999元/月，企业版按需定制。",
     {"source": "产品文档", "category": "产品", "tags": ["产品A", "定价"]}),
]

# 批量添加
for content, meta in hr_docs + it_docs + product_docs:
    kb.add_document(content, meta)

print(f'✅ 知识库初始化完成')
print(json.dumps(kb.get_stats(), indent=2, ensure_ascii=False))

## 3. 测试问答

In [ ]:
# 测试问题
questions = [
    "我有多少天年假？",
    "忘记密码怎么办？",
    "出差报销标准是什么？",
    "产品A多少钱？"
]

for q in questions:
    print(f'\n❓ 问题: {q}')
    result = kb.ask(q)
    print(f'📚 来源: {result["sources"]}')
    print(f'📝 上下文长度: {result["context_length"]} 字符')

## 4. 按类别搜索

In [ ]:
# 只搜索 IT 类别
print('🔍 搜索 IT 类别: "怎么连接公司网络"\n')
results = kb.search("怎么连接公司网络", top_k=3, category="IT")

for i, r in enumerate(results):
    print(f'{i+1}. [{r["metadata"]["source"]}] (相似度: {r["score"]:.3f})')
    print(f'   {r["content"][:60]}...')

## 5. 部署建议

In [ ]:
deployment_guide = """
🚀 企业知识库部署指南 by SherryAGI

1️⃣ 开发环境
   - 向量库: Chroma (嵌入式)
   - LLM: OpenAI API
   - 框架: LangChain

2️⃣ 生产环境 (小规模 <10万文档)
   - 向量库: pgvector (PostgreSQL 扩展)
   - LLM: Azure OpenAI (瑞士节点)
   - 部署: Docker + Kubernetes

3️⃣ 生产环境 (大规模 >100万文档)
   - 向量库: Milvus / Qdrant
   - LLM: 本地部署 (Llama / Qwen)
   - 部署: 分布式集群

4️⃣ 瑞士合规部署
   - 数据驻留: Azure Switzerland North
   - 加密: AES-256 静态加密
   - 审计: 完整操作日志
   - 合规: FADP, GDPR

5️⃣ 推荐开源方案
   - RAGFlow: 专业 RAG 引擎
   - Dify: 低代码 AI 平台
   - AnythingLLM: 一体化桌面应用
"""

print(deployment_guide)

## 6. API 接口示例

In [ ]:
# FastAPI 接口示例
api_code = '''
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI(title="企业知识库 API by SherryAGI")

class Question(BaseModel):
    query: str
    category: str = None
    top_k: int = 3

@app.post("/ask")
async def ask(q: Question):
    """问答接口"""
    results = kb.search(q.query, q.top_k, q.category)
    return {
        "question": q.query,
        "results": results
    }

@app.post("/add")
async def add_doc(content: str, metadata: dict = None):
    """添加文档"""
    doc_id = kb.add_document(content, metadata)
    return {"id": doc_id, "status": "success"}

@app.get("/stats")
async def get_stats():
    """获取统计"""
    return kb.get_stats()
'''

print('📡 FastAPI 接口示例:')
print(api_code)

---
**By SherryAGI** | [DigitalTrans](https://github.com/AIB612/DigitalTrans)